In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [0]:
from google.colab import files
import glob

In [0]:
logs = glob.glob('models/resnet/*True_*.log')
#print(logs)
#for log in logs:
    #files.download(log)
files.download('models/resnet/small_resnet_crossentropy_lr0.001_wd0_adapTrue_14.log')    

In [3]:
!unzip "/content/drive/My Drive/project/fer.zip"

Archive:  /content/drive/My Drive/project/fer.zip
   creating: fer/
   creating: fer/PublicTest/
  inflating: fer/PublicTest/fer31145.png  
  inflating: fer/PublicTest/fer31623.png  
  inflating: fer/PublicTest/fer29078.png  
  inflating: fer/PublicTest/fer31637.png  
  inflating: fer/PublicTest/fer30529.png  
  inflating: fer/PublicTest/fer31151.png  
  inflating: fer/PublicTest/fer30267.png  
  inflating: fer/PublicTest/fer31179.png  
  inflating: fer/PublicTest/fer29736.png  
  inflating: fer/PublicTest/fer29050.png  
  inflating: fer/PublicTest/fer30501.png  
  inflating: fer/PublicTest/fer29044.png  
  inflating: fer/PublicTest/fer30515.png  
  inflating: fer/PublicTest/fer30273.png  
  inflating: fer/PublicTest/fer29722.png  
  inflating: fer/PublicTest/fer31186.png  
  inflating: fer/PublicTest/fer29911.png  
  inflating: fer/PublicTest/fer30298.png  
  inflating: fer/PublicTest/fer31838.png  
  inflating: fer/PublicTest/fer31192.png  
  inflating: fer/PublicTest/fer29905.png  


In [1]:
!ls

sample_data


In [5]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 31kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58622000 @  0x7f3ad3a111c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 14.9MB/s 
    100% |████████████████████████████████| 2.0MB 4.3MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [6]:
!pip3 install Pillow==4.0.0

    100% |████████████████████████████████| 5.6MB 4.6MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
import os
import numpy as np
from tqdm import tqdm
import logging
import time
import pickle
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.environ['OMP_NUM_THREADS'] = str(os.cpu_count())
print('{} CPUs available!'.format(os.cpu_count()))
print('GPU is {}!'.format(torch.cuda.get_device_name(torch.cuda.current_device())))

2 CPUs available!
GPU is Tesla K80!


In [0]:
def init_logger(log_path, to_console=False):
    """Init the logger to log info in terminal and file `log_path`.

    In general, it is useful to have a logger so that every output to the
    terminal is saved in a permanent file.

    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter(
            '%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        if to_console:
            stream_handler = logging.StreamHandler()
            stream_handler.setFormatter(logging.Formatter('%(message)s'))
            logger.addHandler(stream_handler)


class RunningAverage(object):

    def __init__(self, window):
        self.window = window
        self.values = []
        self.mean = 0

    def update(self, value):
        self.values.append(value)
        if len(self.values) > self.window:
            self.mean += (value - self.values.pop(0)) / self.window
        else:
            self.mean = sum(self.values) / len(self.values)

    def __call__(self):
        return self.mean

      
class FerDataset(Dataset):

    def __init__(self, base_path, data='fer', mode='train',
                 label='fer_emotion', only_emotions=False, transform=None):
        """Define FER-specific dataset.

        Args:
            base_path (string): path to base folder containing output of create_dataset.py
            data (string): one of {'fer', 'ferplus'}
            mode (string): one of {'train', 'eval', 'test'}
            label (string): one of {'fer_emotion', 'ferplus_max', 'ferplus_votes'}
            only_emotions (bool): If True, will discard of classes 'unknown' and 'not-a-face'
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Sanity checks
        if data not in ['fer', 'ferplus']:
            raise ValueError('Choose one of {"fer", "ferplus"} as data.')
        if mode not in ['train', 'eval', 'test']:
            raise ValueError('Choose one of {"train", "eval", "test"} as mode.')
        if label not in ['fer_emotion', 'ferplus_max', 'ferplus_votes']:
            raise ValueError('Choose one of {"fer_emotion", "ferplus_max", "ferplus_votes"} as label.')

        self.base = base_path
        if transform is not None:
            raise NotImplementedError('Additional transforms beside normalization unsupported as of yet.')

        # Load mean and std for normalization
        self.norm_mean = self._load_pickle(os.path.join(self.base, 'mean_Training.pkl'))
        self.norm_std = self._load_pickle(os.path.join(self.base, 'std_Training.pkl'))

        # Read the label info
        self.image_names = []
        self.labels = None
        self.n_classes = 7 if label == 'fer_emotion' else 10
        self._prepare_data(data, mode, label, only_emotions)
        self.data_len = len(self.labels)

    @staticmethod
    def _load_pickle(path):
        with open(path, 'rb') as fp:
            content = pickle.load(fp)
        return content

    def _prepare_data(self, data, mode, label, only_emotions=False):

        # Load labels from pickle
        modes = {'train': 'Training', 'eval': 'PublicTest', 'test': 'PrivateTest'}
        mode = modes[mode]
        info_list = self._load_pickle(
            os.path.join(self.base, '{}_{}.pkl'.format(data, mode)))

        # Prepare labels
        emotions = []
        votes_all = []
        votes_max = []
        cut = 8 if only_emotions else 10

        for image_name, emotion, votes in info_list:
            argmax = np.argmax(votes)
            if argmax > (cut - 1):
                continue
            self.image_names.append(os.path.join(self.base, mode, image_name))
            emotions.append(emotion)
            votes_all.append(votes[:cut])
            votes_max.append(argmax)


        self.image_names = np.array(self.image_names)
        self.data_len = len(info_list)

        #onehot = np.zeros((self.data_len, self.n_classes))
        #indices = emotions if label == 'fer_emotion' else votes_max
        #onehot[np.arange(self.data_len), indices] = 1.

        votes_all = np.array(votes_all, dtype=np.float) / 10.

        if label == 'ferplus_votes':
            self.labels = torch.FloatTensor(votes_all)
        elif label == 'fer_emotion':
            self.labels = torch.LongTensor(emotions)
        else:
            self.labels = torch.LongTensor(votes_max)

    def __getitem__(self, index):
        # Load and preprocess image
        image_name = self.image_names[index]
        image = Image.open(image_name)
        image = (image - self.norm_mean) / self.norm_std
        image = torch.FloatTensor(image).expand(1, -1, -1)

        # Get label
        label = self.labels[index]

        return (image, label)

    def __len__(self):
        return self.data_len

In [0]:
class GatingUnit(nn.Module):
    """Gating unit described in 'Convolutional Networks with Adaptive Inference Graphs'."""

    def __init__(self, in_channels, gate_dim, tau=0.8, bias=True):
        super(GatingUnit, self).__init__()

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.estimate_relevance = nn.Sequential(
            nn.Linear(in_channels, gate_dim, bias=bias),
            nn.ReLU(),
            nn.Linear(gate_dim, 2, bias=bias))
        self.tau = tau

    def forward(self, x):
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.estimate_relevance(x)
        x = F.gumbel_softmax(x, tau=self.tau, hard=True)
        return x


class AdaptiveConv2d(nn.Module):
    """Adaptive Conv2d layer described in 'Convolutional Networks with Adaptive Inference Graphs'."""

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True, gate_dim=16, tau=1.0, device=None):
        super(AdaptiveConv2d, self).__init__()

        self.gate = GatingUnit(in_channels, gate_dim, tau=tau)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride,
                              padding, dilation, groups, bias)
        
        self.device = device
        self.samples_in = torch.tensor(0, dtype=torch.float, device=device)
        self.samples_out = torch.tensor(0, dtype=torch.float, device=device)
        
    def get_exec_rate(self):
        if self.samples_in > 0:
            return self.samples_out / self.samples_in
        else:
            return torch.tensor(1, dtype=torch.float, device=device)
    
    def reset_exec_rate(self):
        self.samples_in.zero_()
        self.samples_out.zero_()

    def forward(self, x):
        decision = self.gate(x)[:, 1]
        self.samples_in += x.size(0)
        self.samples_out += decision.sum().item()
        x = self.conv(x)
        x.transpose_(0, -1).mul_(decision).transpose_(0, -1)
        return x


def conv3x3(in_planes, out_planes, stride=1, adaptive=False, device=None):
    """3x3 convolution with padding"""
    if not adaptive:
        return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                         padding=1, bias=False)
    else:
        return AdaptiveConv2d(in_planes, out_planes, kernel_size=3,
                              stride=stride, padding=1, bias=False, device=device)


class Block(nn.Module):

    def __init__(self, in_planes, out_planes, stride=1, adaptive=False, batchnorm=False, device=None):
        super(Block, self).__init__()

        self.downsample = None
        self.device = None

        if batchnorm:
            self.layer1 = nn.Sequential(
                conv3x3(in_planes, out_planes, stride, adaptive, device=device),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_planes))

            self.layer2 = nn.Sequential(
                conv3x3(out_planes, out_planes),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_planes))

            if stride != 1 or in_planes != out_planes:
                self.downsample = nn.Sequential(
                    nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm2d(out_planes))

        else:
            self.layer1 = nn.Sequential(
                conv3x3(in_planes, out_planes, stride, adaptive),
                nn.ReLU(inplace=True))

            self.layer2 = nn.Sequential(
                conv3x3(out_planes, out_planes),
                nn.ReLU(inplace=True))

            if stride != 1 or in_planes != out_planes:
                self.downsample = nn.Conv2d(in_planes, out_planes,
                    kernel_size=1, stride=stride, bias=False)
                
    def get_exec_rate(self):
        if isinstance(self.layer1[0], AdaptiveConv2d):
            return self.layer1[0].get_exec_rate()
        else:
            return torch.FloatTensor([1], device=self.device)
        
    def reset_exec_rate(self):
        if isinstance(self.layer1[0], AdaptiveConv2d):
            return self.layer1[0].reset_exec_rate()
        else:
            pass
    
    def forward(self, x):
        residual = x

        out = self.layer1(x)
        out = self.layer2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        return out


class ResNet(nn.Module):

    def __init__(self, num_classes=10, adaptive=False, batchnorm=False, device=None):
        super(ResNet, self).__init__()

        self.block0 = Block(1, 64, stride=2, adaptive=False, batchnorm=batchnorm)
        self.block1 = Block(64, 128, stride=2, adaptive=False, batchnorm=batchnorm)
        self.block2 = Block(128, 256, stride=2, adaptive=False, batchnorm=batchnorm)
        self.block3 = Block(256, 256, stride=2, adaptive=adaptive, batchnorm=batchnorm, device=device)
        self.block4 = Block(256, 256, stride=1, adaptive=adaptive, batchnorm=batchnorm, device=device)
        self._blocks = [self.block3, self.block4]

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, num_classes)
        self.device = device

        self.reset_parameters()

    def reset_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    def get_exec_rates(self):
        return [block.get_exec_rate() for block in self._blocks]
    
    def reset_exec_rates(self):
        for block in self._blocks:
            block.reset_exec_rate()

    def forward(self, x):
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


In [5]:
base_path = 'fer' 
model_dir = 'models/resnet'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
i = 16
hparams = {}
hparams['n_epochs'] = 50
hparams['data'] = 'ferplus'
hparams['label'] = 'ferplus_max'
hparams['batch_size'] = 64
hparams['wd'] = 0
hparams['lr'] = 1e-3
hparams['adaptive'] = True
hparams['target_rate'] = 0.9
hparams['rate_loss_factor'] = 1
hparams['batchnorm'] = True
hparams['only_emotions'] = False
hparams['scheduler'] = 'StepLR' # 'StepLR' or 'OnPlateau'
hparams['scheduler_patience'] = 4
hparams['scheduler_factor'] = 0.25
hparams['scheduler_step_size'] = 20
hparams['scheduler_gamma'] = 0.25
hparams['loss'] = 'crossentropy'
hparams['amsgrad'] = False
hparams['fix_wd'] = False


# Prepare dataloaders
data = hparams['data']
label = hparams['label']
batch_size = hparams['batch_size']

train_dataset = FerDataset(base_path=base_path, data=data, mode='train', label=label, only_emotions=hparams['only_emotions'])
eval_dataset = FerDataset(base_path=base_path, data=data, mode='eval', label=label, only_emotions=hparams['only_emotions'])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Prepare the network
model = ResNet(num_classes=train_dataset.n_classes, adaptive=hparams['adaptive'], batchnorm=hparams['batchnorm'], device=device)
log_softmax = nn.LogSoftmax(dim=-1)
cross_entropy_loss = nn.CrossEntropyLoss(reduction='elementwise_mean')
kldiv_loss = nn.KLDivLoss(reduction='sum')

if len(train_dataset.labels.shape) == 1:
    criterion = cross_entropy_loss
    def sum_correct(log_probs, labels):
        return log_probs.argmax(dim=-1).eq(labels).sum().item()
else:
    criterion = kldiv_loss
    def sum_correct(log_probs, labels):
        return log_probs.argmax(dim=-1).eq(labels.argmax(dim=-1)).sum().item()
    
wd = 0 if hparams['fix_wd'] else hparams['wd']
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['lr'], weight_decay=hparams['wd'], amsgrad=hparams['amsgrad'])

if hparams['scheduler'] == 'StepLR':
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hparams['scheduler_step_size'], gamma=hparams['scheduler_gamma'])
else:
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=hparams['scheduler_factor'], patience=hparams['scheduler_patience'])



hparams['eval_steps'] = len(train_dataloader)


log_path = os.path.join(model_dir, 'small_resnet_{}_lr{}_wd{}_adap{}_{}.log'.format(
    hparams['loss'], hparams['lr'], hparams['wd'], hparams['adaptive'], i))
init_logger(log_path, to_console=False)

logging.info('### Model ###\n' + model.__repr__())
logging.info('### Optimizer ###\n' + optimizer.__repr__())

hparams_str = " ; ".join("{}: {}".format(k, v) for k, v in hparams.items())
logging.info('### HParams ###\n' + hparams_str)


n_epochs = hparams['n_epochs']
eval_steps = hparams['eval_steps']
n_batches = len(train_dataloader)
wd = hparams['wd']

checkpoint_dir = os.path.join(model_dir, 'checkpoints')
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [7]:
model.to(device)
target_rate = hparams['target_rate']
step = 0
loss_avg = RunningAverage(window=20)
rate1 = RunningAverage(window=20)
rate2 = RunningAverage(window=20)
model.train()
with tqdm(total=n_epochs * n_batches) as t:
    train_samples_correct = 0
    train_samples = 0
    for epoch in range(n_epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            if hparams['adaptive']:
                model.reset_exec_rates()
            logits = model(x_batch)
            log_probs = log_softmax(logits)
            loss = criterion(log_probs, y_batch)
            
            if hparams['adaptive']:
                exec_rates = model.get_exec_rates()
                rate_loss = (exec_rates[0] - target_rate)**2 + (exec_rates[1] - target_rate)**2
                loss += hparams['rate_loss_factor'] * rate_loss

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            train_samples += batch_size
            train_samples_correct += sum_correct(log_probs, y_batch)
            acc = train_samples_correct / train_samples * 100
            loss_avg.update(loss.item())
            rate1_batch = exec_rates[0].item()
            rate2_batch = exec_rates[1].item()
            rate1.update(rate1_batch)
            rate2.update(rate2_batch)
            logging.info('[TRAIN] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}; Rate1: {:02.3f}; Rate2: {:02.3f}'.format(step, loss, acc, rate1_batch, rate2_batch))
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), acc='{:02.3f}'.format(acc), rate1='{:02.2f}'.format(rate1()), rate2='{:02.2f}'.format(rate2()))
            t.update()

            step += 1


            # Evaluate
            if step > 0 and step % eval_steps == 0:
                model.eval()
                eval_samples_correct = 0
                eval_loss = 0
                with torch.no_grad():
                    for x_batch, y_batch in eval_dataloader:
                        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                        logits = model(x_batch)
                        log_probs = log_softmax(logits)
                        eval_loss += criterion(log_probs, y_batch)
                        eval_samples_correct += sum_correct(log_probs, y_batch)
                        optimizer.zero_grad()

                eval_loss /= len(eval_dataloader)
                eval_acc = eval_samples_correct / len(eval_dataset) * 100
                eval_summary = '[EVAL] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, eval_loss, eval_acc)
                if hparams['scheduler'] == 'StepLR':
                    scheduler.step()
                else:
                    scheduler.step(eval_loss)
                logging.info(eval_summary)
                t.write(eval_summary)
                checkpoint = {'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'step': step,
                              'eval_loss': eval_loss,
                              'eval_acc': eval_acc,
                              'train_run_loss': loss_avg(),
                              'train_acc' : acc,
                              'hparams': hparams}
                filename = '{}_{}_{}.pth.tar'.format(time.strftime("%Y%m%d-%H%M%S"), step, i)
                torch.save(checkpoint, os.path.join(checkpoint_dir, filename))
                model.train()

  2%|▏         | 446/22300 [00:45<34:59, 10.41it/s, acc=40.562, loss=1.904, rate1=0.58, rate2=0.34]

[EVAL] Step: 446 ; Loss: 1.449 ; Acc: 50.712


  4%|▍         | 893/22300 [01:30<2:28:51,  2.40it/s, acc=46.796, loss=1.769, rate1=0.51, rate2=0.33]

[EVAL] Step: 892 ; Loss: 1.311 ; Acc: 56.161


  6%|▌         | 1339/22300 [02:15<2:26:02,  2.39it/s, acc=51.147, loss=1.553, rate1=0.53, rate2=0.40]

...

[EVAL] Step: 21408 ; Loss: 1067.529 ; Acc: 70.970


 98%|█████████▊| 21855/22300 [36:05<03:03,  2.42it/s, acc=92.771, loss=1.425, rate1=0.00, rate2=0.12]

[EVAL] Step: 21854 ; Loss: 1097.672 ; Acc: 69.992


100%|██████████| 22300/22300 [36:48<00:00, 10.78it/s, acc=92.914, loss=1.439, rate1=0.00, rate2=0.11]

[EVAL] Step: 22300 ; Loss: 3.571 ; Acc: 71.081


In [7]:
model.to(device)
step = 0
loss_avg = RunningAverage(window=20)
model.train()
with tqdm(total=n_epochs * n_batches) as t:
    train_samples_correct = 0
    train_samples = 0
    for epoch in range(n_epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            logits = model(x_batch)
            log_probs = log_softmax(logits)
            loss = criterion(log_probs, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            train_samples += batch_size
            train_samples_correct += sum_correct(log_probs, y_batch)
            acc = train_samples_correct / train_samples * 100
            loss_avg.update(loss.item())
            logging.info('[TRAIN] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, loss, acc))
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), acc='{:02.3f}'.format(acc))
            t.update()

            step += 1


            # Evaluate
            if step > 0 and step % eval_steps == 0:
                model.eval()
                eval_samples_correct = 0
                eval_loss = 0
                with torch.no_grad():
                    for x_batch, y_batch in eval_dataloader:
                        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                        logits = model(x_batch)
                        log_probs = log_softmax(logits)
                        eval_loss += criterion(log_probs, y_batch)
                        eval_samples_correct += sum_correct(log_probs, y_batch)
                        optimizer.zero_grad()

                eval_loss /= len(eval_dataloader)
                eval_acc = eval_samples_correct / len(eval_dataset) * 100
                eval_summary = '[EVAL] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, eval_loss, eval_acc)
                if hparams['scheduler'] == 'StepLR':
                    scheduler.step()
                else:
                    scheduler.step(eval_loss)
                logging.info(eval_summary)
                t.write(eval_summary)
                checkpoint = {'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'step': step,
                              'eval_loss': eval_loss,
                              'eval_acc': eval_acc,
                              'train_run_loss': loss_avg(),
                              'train_acc' : acc,
                              'hparams': hparams}
                filename = '{}_{}_{}.pth.tar'.format(time.strftime("%Y%m%d-%H%M%S"), step, i)
                torch.save(checkpoint, os.path.join(checkpoint_dir, filename))
                model.train()

  3%|▎         | 446/13380 [00:42<19:32, 11.03it/s, acc=37.045, loss=1.684]

[EVAL] Step: 446 ; Loss: 1.599 ; Acc: 40.123


  7%|▋         | 892/13380 [01:24<18:35, 11.20it/s, acc=42.728, loss=1.392]

[EVAL] Step: 892 ; Loss: 1.374 ; Acc: 51.579


 10%|█         | 1338/13380 [02:06<17:59, 11.15it/s, acc=47.615, loss=1.239]

...

[EVAL] Step: 12488 ; Loss: 1.717 ; Acc: 74.406


 97%|█████████▋| 12934/13380 [20:02<00:39, 11.25it/s, acc=89.122, loss=0.002]

[EVAL] Step: 12934 ; Loss: 1.724 ; Acc: 74.574


100%|██████████| 13380/13380 [20:43<00:00, 11.08it/s, acc=89.482, loss=0.001]

[EVAL] Step: 13380 ; Loss: 1.706 ; Acc: 74.239


In [10]:
# best not adaptive: 20181007-102749_5352_1.pth.tar, 20181007-084027.pth.tar, 20181007-114017_5352_2.pth.tar
# best adaptive: 20181007-131608_10258_2.pth.tar, 20181007-165713_10258_13.pth.tar, 20181007-175800_6244_14.pth.tar

checkpoint_dir = 'models/resnet/checkpoints'
checkpoint = torch.load(os.path.join(checkpoint_dir, '20181007-175800_6244_14.pth.tar'))
eval_acc = checkpoint['eval_acc']
model.to(device)
model.load_state_dict(checkpoint['model'])
model.eval()

test_dataset = FerDataset(base_path=base_path, data=data, mode='test', label=label, only_emotions=hparams['only_emotions'])
test_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

test_samples_correct = 0
with torch.no_grad():
    for x_batch, y_batch in test_dataloader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        logits = model(x_batch)
        log_probs = log_softmax(logits)
        test_samples_correct += sum_correct(log_probs, y_batch)

test_acc = test_samples_correct / len(test_dataset) * 100
test_summary = '[TEST] Eval_Acc: {:02.3f} Test_Acc: {:02.3f}'.format(eval_acc, test_acc)
print(test_summary)

[TEST] Eval_Acc: 71.053 Test_Acc: 71.181


In [0]:
files.download('models/resnet/checkpoints/20181007-175800_6244_14.pth.tar')

20181007-083258.pth.tar		 20181007-134753_4906_3.pth.tar
20181007-083339.pth.tar		 20181007-134838_5352_3.pth.tar
20181007-083420.pth.tar		 20181007-134923_5798_3.pth.tar
20181007-083501.pth.tar		 20181007-135008_6244_3.pth.tar
20181007-083541.pth.tar		 20181007-135053_6690_3.pth.tar
20181007-083622.pth.tar		 20181007-135549_446_5.pth.tar
20181007-083703.pth.tar		 20181007-135634_892_5.pth.tar
20181007-083744.pth.tar		 20181007-135719_1338_5.pth.tar
20181007-083825.pth.tar		 20181007-135804_1784_5.pth.tar
20181007-083906.pth.tar		 20181007-135849_2230_5.pth.tar
20181007-083947.pth.tar		 20181007-135934_2676_5.pth.tar
20181007-084027.pth.tar		 20181007-140019_3122_5.pth.tar
20181007-084108.pth.tar		 20181007-140104_3568_5.pth.tar
20181007-084149.pth.tar		 20181007-140149_4014_5.pth.tar
20181007-084230.pth.tar		 20181007-140233_4460_5.pth.tar
20181007-084311.pth.tar		 20181007-140318_4906_5.pth.tar
20181007-084351.pth.tar		 20181007-140404_5352_5.pth.tar
20181007-084432.pth.tar		 2018100

In [20]:
model.to(device)
step = 0
loss_avg = RunningAverage(window=20)
model.train()
with tqdm(total=n_epochs * n_batches) as t:
    train_samples_correct = 0
    train_samples = 0
    for epoch in range(n_epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            logits = model(x_batch)
            log_probs = log_softmax(logits)
            loss = criterion(log_probs, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            train_samples += batch_size
            train_samples_correct += sum_correct(log_probs, y_batch)
            acc = train_samples_correct / train_samples * 100
            loss_avg.update(loss.item())
            logging.info('[TRAIN] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, loss, acc))
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), acc='{:02.3f}'.format(acc))
            t.update()

            step += 1


            # Evaluate
            if step > 0 and step % eval_steps == 0:
                model.eval()
                eval_samples_correct = 0
                eval_loss = 0
                for x_batch, y_batch in eval_dataloader:
                    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                    logits = model(x_batch)
                    log_probs = log_softmax(logits)
                    eval_loss += criterion(log_probs, y_batch)
                    eval_samples_correct += sum_correct(log_probs, y_batch)

                eval_loss /= len(eval_dataloader)
                eval_acc = eval_samples_correct / len(eval_dataset) * 100
                eval_summary = '[EVAL] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, eval_loss, eval_acc)
                if hparams['scheduler'] == 'StepLR':
                    scheduler.step()
                else:
                    scheduler.step(eval_loss)
                logging.info(eval_summary)
                t.write(eval_summary)
                checkpoint = {'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'step': step,
                              'eval_loss': eval_loss,
                              'eval_acc': eval_acc,
                              'train_run_loss': loss_avg(),
                              'train_acc' : acc,
                              'hparams': hparams}
                filename = time.strftime("%Y%m%d-%H%M%S") + '.pth.tar'
                torch.save(checkpoint, os.path.join(checkpoint_dir, filename))
                model.train()

  1%|          | 1191/119000 [01:01<1:37:22, 20.16it/s, acc=45.816, loss=1.443]

[EVAL] Step: 1190 ; Loss: 1.471 ; Acc: 50.992


  2%|▏         | 2381/119000 [02:03<9:04:25,  3.57it/s, acc=52.148, loss=1.174]

[EVAL] Step: 2380 ; Loss: 1.256 ; Acc: 58.815


  3%|▎         | 3572/119000 [03:04<9:02:50,  3.54it/s, acc=56.315, loss=0.904]

...

[EVAL] Step: 71400 ; Loss: 2.276 ; Acc: 74.322


 61%|██████    | 72591/119000 [1:02:32<3:40:56,  3.50it/s, acc=95.265, loss=0.000]

[EVAL] Step: 72590 ; Loss: 2.270 ; Acc: 74.322


 62%|██████▏   | 73612/119000 [1:03:22<37:25, 20.21it/s, acc=95.330, loss=0.000]


KeyboardInterrupt: ignored

In [22]:
model.to(device)
step = 0
loss_avg = RunningAverage(window=20)
model.train()
with tqdm(total=n_epochs * n_batches) as t:
    train_samples_correct = 0
    train_samples = 0
    for epoch in range(n_epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            logits = model(x_batch)
            log_probs = log_softmax(logits)
            loss = criterion(log_probs, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            train_samples += batch_size
            train_samples_correct += sum_correct(log_probs, y_batch)
            acc = train_samples_correct / train_samples * 100
            loss_avg.update(loss.item())
            logging.info('[TRAIN] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, loss, acc))
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), acc='{:02.3f}'.format(acc))
            t.update()

            step += 1


            # Evaluate
            if step > 0 and step % eval_steps == 0:
                model.eval()
                eval_samples_correct = 0
                eval_loss = 0
                for x_batch, y_batch in eval_dataloader:
                    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                    logits = model(x_batch)
                    log_probs = log_softmax(logits)
                    eval_loss += criterion(log_probs, y_batch)
                    eval_samples_correct += sum_correct(log_probs, y_batch)

                eval_loss /= len(eval_dataloader)
                eval_acc = eval_samples_correct / len(eval_dataset) * 100
                eval_summary = '[EVAL] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, eval_loss, eval_acc)
                if hparams['scheduler'] == 'StepLR':
                    scheduler.step()
                else:
                    scheduler.step(eval_loss)
                logging.info(eval_summary)
                t.write(eval_summary)
                checkpoint = {'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'step': step,
                              'eval_loss': eval_loss,
                              'eval_acc': eval_acc,
                              'train_run_loss': loss_avg(),
                              'train_acc' : acc,
                              'hparams': hparams}
                filename = time.strftime("%Y%m%d-%H%M%S") + '.pth.tar'
                torch.save(checkpoint, os.path.join(checkpoint_dir, filename))
                model.train()

  1%|          | 223/22300 [00:40<1:02:06,  5.92it/s, acc=44.626, loss=1.457]

[EVAL] Step: 223 ; Loss: 1.425 ; Acc: 52.193


  2%|▏         | 446/22300 [01:20<1:02:46,  5.80it/s, acc=53.274, loss=1.168]

[EVAL] Step: 446 ; Loss: 1.317 ; Acc: 56.468


  3%|▎         | 669/22300 [01:59<1:01:45,  5.84it/s, acc=59.208, loss=0.973]

...

[EVAL] Step: 5575 ; Loss: 1.881 ; Acc: 68.120


 26%|██▌       | 5798/22300 [17:14<46:58,  5.85it/s, acc=91.307, loss=0.005]

[EVAL] Step: 5798 ; Loss: 1.903 ; Acc: 68.092


 27%|██▋       | 6021/22300 [17:52<45:47,  5.93it/s, acc=91.627, loss=0.012]


KeyboardInterrupt: ignored

In [28]:
model.to(device)
step = 0
loss_avg = RunningAverage(window=20)
model.train()
with tqdm(total=n_epochs * n_batches) as t:
    train_samples_correct = 0
    train_samples = 0
    for epoch in range(n_epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            logits = model(x_batch)
            log_probs = log_softmax(logits)
            loss = criterion(log_probs, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            train_samples += batch_size
            train_samples_correct += sum_correct(log_probs, y_batch)
            acc = train_samples_correct / train_samples * 100
            loss_avg.update(loss.item())
            logging.info('[TRAIN] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, loss, acc))
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), acc='{:02.3f}'.format(acc))
            t.update()

            step += 1


            # Evaluate
            if step > 0 and step % eval_steps == 0:
                model.eval()
                eval_samples_correct = 0
                eval_loss = 0
                for x_batch, y_batch in eval_dataloader:
                    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                    logits = model(x_batch)
                    log_probs = log_softmax(logits)
                    eval_loss += criterion(log_probs, y_batch)
                    eval_samples_correct += sum_correct(log_probs, y_batch)

                eval_loss /= len(eval_dataloader)
                eval_acc = eval_samples_correct / len(eval_dataset) * 100
                eval_summary = '[EVAL] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, eval_loss, eval_acc)
                if hparams['scheduler'] == 'StepLR':
                    scheduler.step()
                else:
                    scheduler.step(eval_loss)
                logging.info(eval_summary)
                t.write(eval_summary)
                checkpoint = {'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'step': step,
                              'eval_loss': eval_loss,
                              'eval_acc': eval_acc,
                              'train_run_loss': loss_avg(),
                              'train_acc' : acc,
                              'hparams': hparams}
                filename = time.strftime("%Y%m%d-%H%M%S") + '.pth.tar'
                torch.save(checkpoint, os.path.join(checkpoint_dir, filename))
                model.train()

  1%|          | 447/44600 [00:40<4:30:31,  2.72it/s, acc=35.375, loss=1.750]

[EVAL] Step: 446 ; Loss: 1.814 ; Acc: 39.844


  2%|▏         | 893/44600 [01:19<4:27:25,  2.72it/s, acc=41.789, loss=1.485]

[EVAL] Step: 892 ; Loss: 1.664 ; Acc: 50.070


  3%|▎         | 1339/44600 [01:59<4:20:37,  2.77it/s, acc=46.202, loss=1.278]

...

[EVAL] Step: 26760 ; Loss: 493957.250 ; Acc: 73.680


 61%|██████    | 27207/44600 [40:17<1:42:47,  2.82it/s, acc=92.786, loss=0.002]

[EVAL] Step: 27206 ; Loss: 8.340 ; Acc: 73.764


 61%|██████    | 27298/44600 [40:25<25:37, 11.26it/s, acc=92.810, loss=0.001]


KeyboardInterrupt: ignored

In [30]:
model.to(device)
step = 0
loss_avg = RunningAverage(window=20)
model.train()
with tqdm(total=n_epochs * n_batches) as t:
    train_samples_correct = 0
    train_samples = 0
    for epoch in range(n_epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            logits = model(x_batch)
            log_probs = log_softmax(logits)
            loss = criterion(log_probs, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            train_samples += batch_size
            train_samples_correct += sum_correct(log_probs, y_batch)
            acc = train_samples_correct / train_samples * 100
            loss_avg.update(loss.item())
            logging.info('[TRAIN] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, loss, acc))
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), acc='{:02.3f}'.format(acc))
            t.update()

            step += 1


            # Evaluate
            if step > 0 and step % eval_steps == 0:
                model.eval()
                eval_samples_correct = 0
                eval_loss = 0
                for x_batch, y_batch in eval_dataloader:
                    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                    logits = model(x_batch)
                    log_probs = log_softmax(logits)
                    eval_loss += criterion(log_probs, y_batch)
                    eval_samples_correct += sum_correct(log_probs, y_batch)

                eval_loss /= len(eval_dataloader)
                eval_acc = eval_samples_correct / len(eval_dataset) * 100
                eval_summary = '[EVAL] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, eval_loss, eval_acc)
                if hparams['scheduler'] == 'StepLR':
                    scheduler.step()
                else:
                    scheduler.step(eval_loss)
                logging.info(eval_summary)
                t.write(eval_summary)
                checkpoint = {'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'step': step,
                              'eval_loss': eval_loss,
                              'eval_acc': eval_acc,
                              'train_run_loss': loss_avg(),
                              'train_acc' : acc,
                              'hparams': hparams}
                filename = time.strftime("%Y%m%d-%H%M%S") + '.pth.tar'
                torch.save(checkpoint, os.path.join(checkpoint_dir, filename))
                model.train()

  1%|          | 446/44600 [00:40<1:03:27, 11.60it/s, acc=38.807, loss=1.675]

[EVAL] Step: 446 ; Loss: 2.101 ; Acc: 36.435


  2%|▏         | 892/44600 [01:20<1:01:35, 11.83it/s, acc=44.639, loss=1.365]

[EVAL] Step: 892 ; Loss: 1.363 ; Acc: 52.585


  3%|▎         | 1338/44600 [02:00<1:01:01, 11.81it/s, acc=49.242, loss=1.145]

...

[EVAL] Step: 43708 ; Loss: 2.407 ; Acc: 73.903


 99%|█████████▉| 44154/44600 [1:05:45<00:37, 11.75it/s, acc=96.183, loss=0.000]

[EVAL] Step: 44154 ; Loss: 2.458 ; Acc: 74.322


100%|██████████| 44600/44600 [1:06:25<00:00, 11.68it/s, acc=96.221, loss=0.000]

[EVAL] Step: 44600 ; Loss: 2.441 ; Acc: 74.267


In [32]:
model.to(device)
step = 0
loss_avg = RunningAverage(window=20)
model.train()
with tqdm(total=n_epochs * n_batches) as t:
    train_samples_correct = 0
    train_samples = 0
    for epoch in range(n_epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            logits = model(x_batch)
            log_probs = log_softmax(logits)
            loss = criterion(log_probs, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            train_samples += batch_size
            train_samples_correct += sum_correct(log_probs, y_batch)
            acc = train_samples_correct / train_samples * 100
            loss_avg.update(loss.item())
            logging.info('[TRAIN] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, loss, acc))
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), acc='{:02.3f}'.format(acc))
            t.update()

            step += 1


            # Evaluate
            if step > 0 and step % eval_steps == 0:
                model.eval()
                eval_samples_correct = 0
                eval_loss = 0
                for x_batch, y_batch in eval_dataloader:
                    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                    logits = model(x_batch)
                    log_probs = log_softmax(logits)
                    eval_loss += criterion(log_probs, y_batch)
                    eval_samples_correct += sum_correct(log_probs, y_batch)

                eval_loss /= len(eval_dataloader)
                eval_acc = eval_samples_correct / len(eval_dataset) * 100
                eval_summary = '[EVAL] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, eval_loss, eval_acc)
                if hparams['scheduler'] == 'StepLR':
                    scheduler.step()
                else:
                    scheduler.step(eval_loss)
                logging.info(eval_summary)
                t.write(eval_summary)
                checkpoint = {'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'step': step,
                              'eval_loss': eval_loss,
                              'eval_acc': eval_acc,
                              'train_run_loss': loss_avg(),
                              'train_acc' : acc,
                              'hparams': hparams}
                filename = time.strftime("%Y%m%d-%H%M%S") + '.pth.tar'
                torch.save(checkpoint, os.path.join(checkpoint_dir, filename))
                model.train()

  1%|          | 446/44600 [00:40<1:03:06, 11.66it/s, acc=38.321, loss=1.599]

[EVAL] Step: 446 ; Loss: 1.767 ; Acc: 43.811


  2%|▏         | 892/44600 [01:20<1:01:32, 11.84it/s, acc=44.032, loss=1.378]

[EVAL] Step: 892 ; Loss: 1.514 ; Acc: 54.820


  3%|▎         | 1338/44600 [01:59<1:01:11, 11.78it/s, acc=48.438, loss=1.210]

...

[EVAL] Step: 43708 ; Loss: 2.167 ; Acc: 75.300


 99%|█████████▉| 44154/44600 [1:05:45<00:37, 11.74it/s, acc=95.290, loss=0.002]

[EVAL] Step: 44154 ; Loss: 2.123 ; Acc: 75.272


100%|██████████| 44600/44600 [1:06:25<00:00, 11.69it/s, acc=95.337, loss=0.000]

[EVAL] Step: 44600 ; Loss: 2.112 ; Acc: 74.993


In [28]:
model.to(device)
step = 0
loss_avg = RunningAverage(window=20)
model.train()
with tqdm(total=n_epochs * n_batches) as t:
    train_samples_correct = 0
    train_samples = 0
    for epoch in range(n_epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            logits = model(x_batch)
            log_probs = log_softmax(logits)
            loss = criterion(log_probs, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            if hparams['fix_wd']:
                for group in optimizer.param_groups:
                    for param in group['params']:
                        param.data = param.data.add(-hparams['wd'] * group['lr'], param.data)
            optimizer.step()

            # Logging
            train_samples += batch_size
            train_samples_correct += sum_correct(log_probs, y_batch)
            acc = train_samples_correct / train_samples * 100
            loss_avg.update(loss.item())
            logging.info('[TRAIN] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, loss, acc))
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), acc='{:02.3f}'.format(acc))
            t.update()

            step += 1


            # Evaluate
            if step > 0 and step % eval_steps == 0:
                model.eval()
                eval_samples_correct = 0
                eval_loss = 0
                for x_batch, y_batch in eval_dataloader:
                    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                    logits = model(x_batch)
                    log_probs = log_softmax(logits)
                    eval_loss += criterion(log_probs, y_batch)
                    eval_samples_correct += sum_correct(log_probs, y_batch)

                eval_loss /= len(eval_dataloader)
                eval_acc = eval_samples_correct / len(eval_dataset) * 100
                eval_summary = '[EVAL] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, eval_loss, eval_acc)
                if hparams['scheduler'] == 'StepLR':
                    scheduler.step()
                else:
                    scheduler.step(eval_loss)
                logging.info(eval_summary)
                t.write(eval_summary)
                checkpoint = {'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'step': step,
                              'eval_loss': eval_loss,
                              'eval_acc': eval_acc,
                              'train_run_loss': loss_avg(),
                              'train_acc' : acc,
                              'hparams': hparams}
                filename = time.strftime("%Y%m%d-%H%M%S") + '.pth.tar'
                torch.save(checkpoint, os.path.join(checkpoint_dir, filename))
                model.train()

  1%|          | 444/44300 [00:41<4:33:46,  2.67it/s, acc=37.145, loss=1.593]

[EVAL] Step: 443 ; Loss: 2.022 ; Acc: 35.266


  2%|▏         | 887/44300 [01:21<4:23:50,  2.74it/s, acc=43.056, loss=1.442]

[EVAL] Step: 886 ; Loss: 1.464 ; Acc: 51.815


  3%|▎         | 1329/44300 [02:01<1:03:06, 11.35it/s, acc=47.351, loss=1.168]

...

[EVAL] Step: 40313 ; Loss: 2.196 ; Acc: 75.148


 92%|█████████▏| 40756/44300 [1:01:57<05:05, 11.59it/s, acc=95.058, loss=0.000]

[EVAL] Step: 40756 ; Loss: 2.214 ; Acc: 74.726


 92%|█████████▏| 40963/44300 [1:02:15<05:04, 10.97it/s, acc=95.082, loss=0.001]


TypeError: ignored

In [19]:
len(train_dataset)

28389

In [21]:
len(train_dataset)

28561

In [9]:
torch.cuda.empty_cache()
model.to(device)
step = 0
loss_avg = RunningAverage(window=20)
model.train()
with tqdm(total=n_epochs * n_batches) as t:
    train_samples_correct = 0
    train_samples = 0
    for epoch in range(n_epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            logits = model(x_batch)
            log_probs = log_softmax(logits)
            loss = criterion(log_probs, y_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            if hparams['fix_wd']:
                for group in optimizer.param_groups:
                    for param in group['params']:
                        param.data = param.data.add(-hparams['wd'] * group['lr'], param.data)
            optimizer.step()

            # Logging
            train_samples += batch_size
            train_samples_correct += sum_correct(log_probs, y_batch)
            acc = train_samples_correct / train_samples * 100
            loss_avg.update(loss.item())
            logging.info('[TRAIN] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, loss, acc))
            t.set_postfix(loss='{:05.3f}'.format(loss_avg()), acc='{:02.3f}'.format(acc))
            t.update()

            step += 1


            # Evaluate
            if step > 0 and step % eval_steps == 0:
                model.eval()
                eval_samples_correct = 0
                eval_loss = 0
                with torch.no_grad():
                    for x_batch, y_batch in eval_dataloader:
                        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                        logits = model(x_batch)
                        log_probs = log_softmax(logits)
                        eval_loss += criterion(log_probs, y_batch)
                        eval_samples_correct += sum_correct(log_probs, y_batch)
                        optimizer.zero_grad()

                eval_loss /= len(eval_dataloader)
                eval_acc = eval_samples_correct / len(eval_dataset) * 100
                eval_summary = '[EVAL] Step: {} ; Loss: {:05.3f} ; Acc: {:02.3f}'.format(step, eval_loss, eval_acc)
                if hparams['scheduler'] == 'StepLR':
                    scheduler.step()
                else:
                    scheduler.step(eval_loss)
                logging.info(eval_summary)
                t.write(eval_summary)
                checkpoint = {'model': model.state_dict(),
                              'optimizer': optimizer.state_dict(),
                              'step': step,
                              'eval_loss': eval_loss,
                              'eval_acc': eval_acc,
                              'train_run_loss': loss_avg(),
                              'train_acc' : acc,
                              'hparams': hparams}
                filename = time.strftime("%Y%m%d-%H%M%S") + '.pth.tar'
                torch.save(checkpoint, os.path.join(checkpoint_dir, filename))
                model.train()

  0%|          | 0/89200 [00:00<?, ?it/s]


RuntimeError: ignored